## 메추리알 카카오맵 크롤링
1. 카카오맵에서 '홍대 맛집' 정보 크롤링
    - 가져와야 할 정보: 이름, 주 메뉴, 위치(위도 경도), 주소, 운영 시간, 별점, 리뷰, 전화번호, 시설정보
    - 1차 가져올 정보: 이름, 위치, 주소, 별점, 리뷰 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt

In [169]:
## 카카오 맵
driver = webdriver.Chrome()
driver.get('https://map.kakao.com/')

In [170]:
## input 창에 검색어 입력
searchbox = driver.find_element(By.ID, 'search.keyword.query')
searchbox.send_keys('홍대 맛집'+ Keys.ENTER)

In [136]:
## 상세보기 리스트
moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')

In [143]:
moreview_element[0].get_attribute('href')

'https://place.map.kakao.com/26235884'

In [145]:
list_url = []
moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
for i in range(4):
    list_url.append(moreview_element[i].get_attribute('href'))
list_url


['https://place.map.kakao.com/26235884',
 'https://place.map.kakao.com/53292857',
 'https://place.map.kakao.com/12771116',
 'https://place.map.kakao.com/400281849']

In [46]:
# 첫번재 상세보기
moreview_element[0].click()

In [ ]:
# 영업시간 더 보기 클릭
driver.find_element(By.CSS_SELECTOR, 'a[class = "btn_more"]').click()

In [181]:
## 웹 브라우저 탭 이동
# 웹 브라우저 탭 정보 출력
driver.window_handles
# driver에서 탭을 이동
driver.switch_to.window(driver.window_handles[1])

#### 매개변수
- name : 매장명
- category : 분류
- star : 종합 별점
- address : 주소
- oper_time : 영업 시간

In [122]:
## 상세page bs4 파싱 분석
soup = bs(driver.page_source, 'html.parser')

In [123]:
## name 이름 스크래핑
soup.find('h2', {'class':'tit_location'}).get_text()

'피오니 홍대점'

In [124]:
## category 분류 스크래핑
soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip()

'카페'

In [125]:
## star 종합 별점 스크래핑
soup.find('span', {'class':'color_b'}).get_text().strip('점')

'3.8'

In [126]:
## address 주소 스크래핑          # 띄어
soup.find('span', {'class': 'txt_address'}).get_text().replace("\n","").replace(" ","")

'서울마포구독막로7길511층(우)04043'

In [133]:
## oper_time 영업 시간 스크래핑
soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip()

'매일 12:00 ~ 21:00\n\n매일 라스트오더 ~ 20:30'

In [120]:
## 영업시간 더보기 나중에 클릭했음. selenium으로 스크래핑

## time 영업시간 스크래핑

# 영업시간 더 보기 클릭
#driver.find_element(By.CSS_SELECTOR, 'a[class = "btn_more"]').click()

# [0] : 상세정보에 나오는 영업시간, [1] : 더보기 누르면 나오는 영업시간
#driver.find_elements(By.CSS_SELECTOR, 'ul[class ="list_operation"]')[1].text.split('\n')

['매일 12:00 ~ 21:00', '매일 라스트오더 ~ 20:30']

In [ ]:
#### 모듈 만들기


In [166]:
data = pd.read_csv('./data.csv')


In [167]:
data

,name,category,star,address,oper_time
0,피오니 홍대점,카페,3.8,서울마포구독막로7길511층(우)04043,매일 12:00 ~ 21:00\n\n매일 라스트오더 ~ 20:30
1,943킹스크로스,테마카페,4.1,"서울마포구양화로16길24지하1층,1~4층(우)04039",월~목 11:30 ~ 21:30\n\n금~일 10:00 ~ 21:30
2,카미야,"돈까스,우동",4.2,서울마포구와우산로21길28-6지하1층(우)04040,매일 11:00 ~ 21:30\n\n매일 라스트오더 ~ 21:00
3,우와 홍대본점,일식,3.4,서울마포구와우산로21길21-162층202호(우)04041,매일 12:00 ~ 22:00\n\n\n\n월~금 휴게시간 14:30 ~ 17:00


In [155]:
soup = bs(driver.page_source, 'html.parser')

In [156]:
soup.find('ul', {'class':'list_operation'}).get_text().strip()

'매일 12:00 ~ 22:00\n\n\n\n월~금 휴게시간 14:30 ~ 17:00'

In [164]:
oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')

In [165]:
oper_more

[<selenium.webdriver.remote.webelement.WebElement (session="16caca5564484e29316bdbe9eeed618f", element="f.8921D724CB5C847707786970A13C338F.d.68A7FCB4C1EACA7AC269318106D31056.e.36")>]

In [189]:
driver.switch_to.window(driver.window_handles[1])

In [191]:
soup = bs(driver.page_source, 'html.parser')

In [192]:
soup

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<title>비스트로주라 | 카카오맵</title>
<meta content="비스트로주라" property="og:title"/>
<meta content="카카오맵" property="og:site_name"/>
<meta content="서울 마포구 와우산로23길 18-7 1층 (서교동 342-16)" property="og:description"/>
<meta content="website" property="og:type"/>
<meta content="http://map3.daum.net/staticmap/og?type=place&amp;srs=wcongnamul&amp;size=400x200&amp;m=483348%2C1126249&amp;service=placeweb" property="og:image"/>
<meta content="https://place.map.kakao.com/27272711" property="og:url"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="비스트로주라" name="twitter:title"/>
<meta content="서울 마포구 와우산로23길 18-7 1층 (서교동 342-16)" name="twitter:description"/>
<meta content="http://map3.daum.net/staticmap/og?type=place&amp;srs=wcongnamul&amp;size=400x200&amp;m=483348%2C1126249&amp;service=placeweb" name="twitter:image"/>
<link href="//t1.daumcdn.net/localimg/localimages/07/common/kakaoma

In [196]:
soup.find_all('span', {'class':'color_b'})[1].text

'120개'